In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%perl 

In [2]:

import os
import re

def read_devnagri_text(text):
    text = text.replace('\u200c', '')
    text = text.replace('\u200d', '')
    text = text.replace('\ufeff', '')
    text = text.replace('\xad', '')
    text = text.replace(u'-','')
    text = text.replace('-','')
    text = text.replace('–','')
    text = text.replace('—','')
    text = text.replace(' : ','')
    text = text.replace('’','')
    text = text.replace('“','')
    text = text.replace('”','')
    text = text.replace('!','')
    text = text.replace('%','')
    text = text.replace('&','')
    text = text.replace('*','')
    text = text.replace('+','')
    text = text.replace('?','')
    text = text.replace('/','')
    text = text.replace(';','')
    text = text.replace('{','')
    text = text.replace('}','')
    text = text.replace('=','')
    text = text.replace(',','')
    text = text.replace('\'','')
    text = text.replace('"','')
    text = text.replace(u'.','')
    text = text.replace('॥','')
    text = text.replace(')','')
    text = text.replace('(','')
    text = text.replace('[','')
    text = text.replace(']','')
    text = text.replace('<','')
    text = text.replace('>','')
    text = text.replace('`','')
    text = text.replace('‘','')
    text = text.replace('#','')
    text = text.replace('।', '')
    text = text.replace('|', '')
    text = re.sub('[0-9a-zA-Z]', '', text)
    text = re.sub('[०-९]', '', text)
    text = text.strip()
    return text


In [ ]:
!pip install indic_transliteration
!pip install devnagri_reader

In [5]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import numpy as np

swaras = ['a', 'A', 'i', 'I', 'u', 'U', 'e', 'E', 'o', 'O', 'f', 'F', 'x', 'X']
vyanjanas = ['k', 'K', 'g', 'G', 'N', 
             'c', 'C', 'j', 'J', 'Y',
             'w', 'W', 'q', 'Q', 'R',
             't', 'T', 'd', 'D', 'n',
             'p', 'P', 'b', 'B', 'm',
             'y', 'r', 'l', 'v','S', 'z', 's', 'h', 'L', '|']
others = ['H', 'Z', 'V', 'M', '~', '/', '\\', '^', '\'']

slp1charlist = swaras + vyanjanas + others

def remove_nonslp1_chars(word):
    newword = ''
    for char in word:
        if char in slp1charlist:
            newword = newword + char
    return newword

def get_sandhi_dataset(datafile):
    word1list = []
    word2list = []
    outputlist = []

    with open(datafile) as fp:
        tests = fp.read().splitlines()
    
    total = 0
    maxlen = 0

    for test in tests:
        #print(test)
    
        if(test.find('=>') == -1):
            continue;
        inout = test.split('=>')
        words = inout[1].split('+')
    
        if(len(words) != 2):
            continue
    
        word1 = words[0].strip()
        word1 = read_devnagri_text(word1)
        slp1word1 = transliterate(word1, sanscript.DEVANAGARI, sanscript.SLP1)
        slp1word1 = remove_nonslp1_chars(slp1word1)
    
        word2 = words[1].strip()
        word2 = read_devnagri_text(word2)
        slp1word2 = transliterate(word2, sanscript.DEVANAGARI, sanscript.SLP1)
        slp1word2 = remove_nonslp1_chars(slp1word2)
    
        expected = inout[0].strip()
        expected = read_devnagri_text(expected)
        slp1expected = transliterate(expected, sanscript.DEVANAGARI, sanscript.SLP1)
        slp1expected = remove_nonslp1_chars(slp1expected)

        if slp1word1 and slp1word2 and slp1expected:
            total = total + 1
        else:
            continue

        fwl = 4
        swl = 2

        if len(slp1word1) > fwl:
            excess = len(slp1word1) - fwl
            slp1word1 = slp1word1[-fwl:]
            slp1expected = slp1expected[excess:]
        if len(slp1word2) > swl:
            excess = len(slp1word2) - swl
            slp1word2 = slp1word2[0:swl]
            slp1expected = slp1expected[:-excess]

        if(maxlen < len(slp1expected)):
            maxlen = len(slp1expected)

        difflen = len(slp1expected) - (len(slp1word1) + len(slp1word2))

        if difflen < 2 and difflen > -3:
            word1list.append(slp1word1)
            word2list.append(slp1word2)
            outputlist.append(slp1expected)

        #print(slp1expected + ' => ' + slp1word1 + ' + ' + slp1word2)
        #print(expected + ' => ' + word1 + ' + ' + word2)

    #print(total)
    #print(maxlen)
    return word1list, word2list, outputlist

def get_xy_data(datafile):
    w1l, w2l, ol = get_sandhi_dataset(datafile)
    return w1l, w2l, ol



In [7]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
import numpy as np
from sklearn.model_selection import train_test_split
#cimport sandhi_data_prepare as sdp

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 16  # Latent dimensionality of the encoding space.

# Vectorize the data.
input_texts = []
target_texts = []
characters = set()

w1l, w2l, ol = get_xy_data("/content/sandhiset.txt")

print("Sandhi dataset created")

for i in range(len(w1l)):
    input_text = w1l[i] + '+' + w2l[i]
    target_text = ol[i]
    # We use "&" as the "start sequence" character for the targets, and "$" as "end sequence" character.
    target_text = '&' + target_text + '$'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in characters:
            characters.add(char)
    for char in target_text:
        if char not in characters:
            characters.add(char)

# Split the training and testing data
input_texts, X_tests, target_texts, Y_tests = train_test_split(input_texts, target_texts, test_size=0.2, random_state=1)

# Using '*' for padding 
characters.add('*')

characters = sorted(list(characters))
num_tokens = len(characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique tokens:', num_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

token_index = dict([(char, i) for i, char in enumerate(characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, token_index[char]] = 1.
    encoder_input_data[i, t + 1:, token_index['*']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, token_index[char]] = 1.
    decoder_input_data[i, t + 1:, token_index['*']] = 1.
    decoder_target_data[i, t:, token_index['*']] = 1.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# Save model
model.save('bis2s.h5')

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, token_index['&']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '$' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

test_file_list = ["/content/Astaadhyaayii.txt"]


for test_file in test_file_list:
    input_texts = []
    target_texts = []
    
    if test_file == "testset":
        input_texts = X_tests
        target_texts = Y_tests
        for i in range(len(target_texts)):
            target_texts[i] = target_texts[i][1:-1]
    else:
        w1l, w2l, ol = get_xy_data(test_file)
    
        for i in range(len(w1l)):
            input_text = w1l[i] + '+' + w2l[i]
            target_text = ol[i]
            input_texts.append(input_text)
            target_texts.append(target_text)
    
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_tokens), dtype='float32')
    
    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text):
            if char not in token_index:
                continue
            encoder_input_data[i, t, token_index[char]] = 1.
        encoder_input_data[i, t + 1:, token_index['*']] = 1.
    
    # Next: inference mode (sampling).
    # Here's the drill:
    # 1) encode input and retrieve initial decoder state
    # 2) run one step of decoder with this initial state
    # and a "start of sequence" token as target.
    # Output will be the next target token
    # 3) Repeat with the current target token and current states
    
    # Define sampling models
    encoder_model = Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(latent_dim*2,))
    decoder_state_input_c = Input(shape=(latent_dim*2,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # Reverse-lookup token index to decode sequences back to something readable.
    reverse_input_char_index = dict((i, char) for char, i in token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in token_index.items())
    
    total = len(encoder_input_data)
    passed = 0
    for seq_index in range(len(encoder_input_data)):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = encoder_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        decoded_sentence = decoded_sentence.strip()
        decoded_sentence = decoded_sentence.strip('$')
        if decoded_sentence == target_texts[seq_index]:
            passed = passed + 1
        """
        else:
            print(str(seq_index)+'/'+str(total))
            print('-')
            print('Input sentence:   ', input_texts[seq_index])
            print('Decoded sentence: ', decoded_sentence)
            print('Expected sentence:', target_texts[seq_index])
        """
    print("\n"+test_file)
    print("Passed: "+str(passed)+'/'+str(total)+', '+str(passed*100/total))

Sandhi dataset created
Number of samples: 64822
Number of unique tokens: 53
Max sequence length for inputs: 7
Max sequence length for outputs: 9
Epoch 1/5
811/811 [==============================] - 18s 16ms/step - loss: 1.8890 - accuracy: 0.4972 - val_loss: 1.3056 - val_accuracy: 0.6429
Epoch 2/5
811/811 [==============================] - 11s 14ms/step - loss: 0.9447 - accuracy: 0.7451 - val_loss: 0.6767 - val_accuracy: 0.8243
Epoch 3/5
811/811 [==============================] - 11s 14ms/step - loss: 0.5208 - accuracy: 0.8635 - val_loss: 0.4181 - val_accuracy: 0.8906
Epoch 4/5
811/811 [==============================] - 12s 14ms/step - loss: 0.3430 - accuracy: 0.9144 - val_loss: 0.2932 - val_accuracy: 0.9271
Epoch 5/5
811/811 [==============================] - 11s 14ms/step - loss: 0.2544 - accuracy: 0.9378 - val_loss: 0.2618 - val_accuracy: 0.9368

/content/Astaadhyaayii.txt
Passed: 580/1574, 36.84879288437103
